In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import navis
from compcon.create_graph import get_neuron_local
import random
import sys
import os
import arkouda as ak
sys.path.insert(0, os.path.abspath("/home/michaelshewarega/Desktop/test/arkouda"))

In [ ]:
# The following code preprocesses the synapses and maps them to the correct segments. 
# Since in our new data structure, neuron segments are represented as nodes, 
# synapses must be mapped to the appropriate segments of each neuron.
# 
# To achieve this, we create a dictionary called `global_mapping`, which stores 
# neuron IDs as keys. For each neuron ID, the dictionary stores the corresponding 
# synapse index and the neuron segment it maps to.

import random
#loading the synapses and the pre and postsynaptic ids of the neurons for each synapse
syn_df = pd.read_csv("path_to/synapses.csv")
pre = syn_df['pre_pt_root_id'].drop_duplicates().values.tolist()
post = syn_df['post_pt_root_id'].drop_duplicates().values.tolist()
syn_ids = list(set(pre + post))
gloabl_mapping = {}

for n_id in ids:
    # Create a consistent random multiplier based on the neuron id
    random.seed(int(n_id), 1000)
    consistent_multiplier = random.randint(1, 1000000000000)
    
    x_temp=[]
    y_temp=[]
    z_temp=[]
    #status of progress
    print((syn_ids.index(n_id)/len(syn_ids))*100, "%")

    #filtered synpases with either pre or post synaptic id being the one of the neuron
    filtered_df = syn_df[(syn_df['pre_pt_root_id'] == n_id) | (syn_df['post_pt_root_id'] == n_id)]

    #indexes of the synapses
    indexes = filtered_df.index.tolist()

    #load current neuron
    cur_neuron = get_neuron_local(n_id)

    #get x, y, z coordinates of neurons
    x_temp.extend((filtered_df.loc[:, "pre_pt_position_x"].values/1000 + filtered_df.loc[:, "post_pt_position_x"].values/1000)/2)
    y_temp.extend((filtered_df.loc[:, "pre_pt_position_y"].values/1000 + filtered_df.loc[:, "post_pt_position_y"].values/1000)/2)
    z_temp.extend((filtered_df.loc[:, "pre_pt_position_z"].values/1000 + filtered_df.loc[:, "post_pt_position_z"].values/1000)/2)
    
    c_coordinates = np.vstack((x_temp, y_temp, z_temp)).T

    #snap synapses to skeleton in neuron
    idloc1, dist1 = cur_neuron.snap(c_coordinates)

    #downsample neuron
    cur_neuron_ds = navis.downsample_neuron(cur_neuron, downsampling_factor=10000, inplace=False)

    #downsample neuron and preserve the skeletons to which the synapses where snapped to
    cur_neuron_ds_preserved = navis.downsample_neuron(cur_neuron, downsampling_factor=10000, preserve_nodes=idloc1, inplace=False)

    #map0 are the skeletons that will map to the neuron segments stored in map1
    map0 = []
    map1 = []
    cur_neuron_ds_array = np.array(cur_neuron_ds.edges)
    
    map0 = map0 + list(cur_neuron_ds_array[:,0])
    map1 = map1 + list(cur_neuron_ds_array[:,0]*consistent_multiplier + cur_neuron_ds_array[:,1])

    map0 = map0 + list(cur_neuron_ds_array[:,1])
    map1 = map1 + list(cur_neuron_ds_array[:,0]*consistent_multiplier + cur_neuron_ds_array[:,1])

    cur_neuron_dsedges=cur_neuron_ds.edges
    cur_neuron_ds = set(map(tuple, cur_neuron_ds.edges))
    cur_neuron_ds_preserved = set(map(tuple, cur_neuron_ds_preserved.edges))
    
    # Find the difference
    difference_set1 = cur_neuron_ds - cur_neuron_ds_preserved
    difference_set2 = cur_neuron_ds_preserved - cur_neuron_ds

    difference_set1 = list(map(list, difference_set1))
    difference_set2 = list(map(list, difference_set2))

    nx_display = nx.Graph()
    nx_display.add_edges_from(difference_set2)
    
    for eds in difference_set1:
        start_node_id = eds[0]
        end_node_id = eds[1]
        
        shortest_path = nx.shortest_path(nx_display, source=start_node_id, target=end_node_id, weight='weight')

        if len(shortest_path) > 2:
            inner_path = shortest_path[1:-1]
            map0.extend(inner_path)
            map1.extend(len(inner_path)*[eds[0]*consistent_multiplier+eds[1]])

    
    mapping0 = dict(zip(map0, map1))
    idloc1 = np.vectorize(mapping0.get)(idloc1).tolist()

    index_to_id_mapping = dict(zip(indexes, idloc1))
    gloabl_mapping[n_id] = index_to_id_mapping

In [ ]:
# This function takes the neurons as input and generates all possible combinations 
# of them. It then maps the corresponding synapses to the appropriate neuron segments 
# using the 'global_mapping' dictionary. Finally, the function returns the synapse 
# information for each combination.
def synaptic_data(neurons):
    synaptic_src, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af, synaptic_dst, x, y, z, n_combo, dists = [], [], [], [], [], [], [], [], [], [], []
    filtered_df = pd.read_csv("/path_to/synapses.csv")
    combinations = filtered_df[['pre_pt_root_id', 'post_pt_root_id']].drop_duplicates().values.tolist()

    for combo in combinations:
            
            x_temp, y_temp, z_temp = [], [], []
            
            df1 = filtered_df[(filtered_df['pre_pt_root_id'] == combo[0]) & (filtered_df['post_pt_root_id'] == combo[1])]
            
            if df1.empty:
                continue
                
            print(combo, (combinations.index(combo)/len(combinations))*100, "%")
            indexes = df1.index.tolist()

            x_temp.extend((df1.loc[:, "pre_pt_position_x"].values/1000 + df1.loc[:, "post_pt_position_x"].values/1000)/2)
            y_temp.extend((df1.loc[:, "pre_pt_position_y"].values/1000 + df1.loc[:, "post_pt_position_y"].values/1000)/2)
            z_temp.extend((df1.loc[:, "pre_pt_position_z"].values/1000 + df1.loc[:, "post_pt_position_z"].values/1000)/2)

            mapping0 = gloabl_mapping[combo[0]]
            mapping1 = gloabl_mapping[combo[1]]
            
            idloc1 = np.vectorize(mapping0.get)(indexes).tolist()
            idloc2 = np.vectorize(mapping1.get)(indexes).tolist()
            
            synaptic_s_bef_temp = np.vectorize(lambda x: neurons[combo[0]]["bef_mapping"].get(x, 0))(idloc1)
            synaptic_s_af_temp = np.vectorize(lambda x: neurons[combo[0]]["af_mapping"].get(x, 0))(idloc1)
            synaptic_d_bef_temp = np.vectorize(lambda x: neurons[combo[1]]["bef_mapping"].get(x, 0))(idloc2)
            synaptic_d_af_temp = np.vectorize(lambda x: neurons[combo[1]]["af_mapping"].get(x, 0))(idloc2)
            
            synaptic_src.extend(idloc1)
            synaptic_s_bef.extend(synaptic_s_bef_temp)
            synaptic_s_af.extend(synaptic_s_af_temp)
            synaptic_dst.extend(idloc2)
            synaptic_d_bef.extend(synaptic_d_bef_temp)
            synaptic_d_af.extend(synaptic_d_af_temp)
            n_combo.extend([(f"{combo[0]}_{combo[1]}")] * len(idloc1))
            x.extend(x_temp)
            y.extend(y_temp)
            z.extend(z_temp)
            dists.extend([0]* len(idloc1))
            
    connection_type_s = ["s"] * len(synaptic_src)
    
    return synaptic_src, synaptic_dst, connection_type_s, x, y, z, n_combo, dists, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af

In [ ]:
# This function takes the edges of a downsampled neuron and the downsampled neuron itself, 
# and constructs a neuron morphology-aware graph, excluding the synapses from the graph creation.
def spatial_connectome_creation(edges, n_ds):
    # Create a consistent random multiplier based on the neuron ID
    random.seed(int(n_ds.name))
    consistent_multiplier = random.randint(1, 1000000000000)
    
    ### create new Graph ### 
    first_elements = edges[:, 0]
    second_elements = edges[:, 1]
    neighbor_matrix = (first_elements[:, np.newaxis] == second_elements[np.newaxis, :])

    # Step 3: Extract neighbor pairs
    neighbor_indices = np.argwhere(neighbor_matrix)
    neighbor_indices = neighbor_indices[neighbor_indices[:, 0] != neighbor_indices[:, 1]]
    neighbor_pairs = edges[neighbor_indices]

    # Step 4: Combine node indices to create unique identifiers for edges
    src_combined = neighbor_pairs[:, 0, 0] * consistent_multiplier + neighbor_pairs[:, 0, 1]
    dst_combined = neighbor_pairs[:, 1, 0] * consistent_multiplier + neighbor_pairs[:, 1, 1]
    
    # Convert to lists for Arkouda
    src = src_combined.astype(np.int64).tolist()
    dst = dst_combined.astype(np.int64).tolist()

    ### get coordinates and calculate distances of nodes ###
    s_bef = neighbor_pairs[:, 0, 0].astype(np.int64).tolist()
    s_af = neighbor_pairs[:, 0, 1].astype(np.int64).tolist()

    d_bef = neighbor_pairs[:, 1, 0].astype(np.int64).tolist()
    d_af = neighbor_pairs[:, 1, 1].astype(np.int64).tolist()

    #mapping
    bef_mapping = dict(zip(src + dst, s_bef + d_bef))
    af_mapping = dict(zip(src + dst, s_af + d_af))

    # Create DataFrame for s_bef and s_af with their corresponding coordinates
    s_bef_coords = n_ds.nodes.set_index('node_id').loc[s_bef][['x', 'y', 'z']].reset_index()
    s_af_coords = n_ds.nodes.set_index('node_id').loc[s_af][['x', 'y', 'z']].reset_index()

    d_bef_coords = n_ds.nodes.set_index('node_id').loc[d_bef][['x', 'y', 'z']].reset_index()
    d_af_coords = n_ds.nodes.set_index('node_id').loc[d_af][['x', 'y', 'z']].reset_index()

    # Ensure lengths match by repeating coordinates for each occurrence
    s_bef_coords_repeated = s_bef_coords.loc[s_bef_coords.index.repeat(s_bef_coords.index.value_counts().reindex(s_bef_coords.index).fillna(1))]
    s_af_coords_repeated = s_af_coords.loc[s_af_coords.index.repeat(s_af_coords.index.value_counts().reindex(s_af_coords.index).fillna(1))]

    d_bef_coords_repeated = d_bef_coords.loc[d_bef_coords.index.repeat(d_bef_coords.index.value_counts().reindex(d_bef_coords.index).fillna(1))]
    d_af_coords_repeated = d_af_coords.loc[d_af_coords.index.repeat(d_af_coords.index.value_counts().reindex(d_af_coords.index).fillna(1))]

    # Extract coordinates as numpy arrays
    x_bef, y_bef, z_bef = s_bef_coords_repeated['x'].values, s_bef_coords_repeated['y'].values, s_bef_coords_repeated['z'].values
    x_af, y_af, z_af = s_af_coords_repeated['x'].values, s_af_coords_repeated['y'].values, s_af_coords_repeated['z'].values

    s_bef_x = x_bef.astype(np.int64).tolist()
    s_bef_y = y_bef.astype(np.int64).tolist()
    s_bef_z = z_bef.astype(np.int64).tolist()
    
    s_af_x = x_af.astype(np.int64).tolist()
    s_af_y = y_af.astype(np.int64).tolist()
    s_af_z = z_af.astype(np.int64).tolist()
    
    s_distances = np.sqrt((x_bef - x_af)**2 + (y_bef - y_af)**2 + (z_bef - z_af)**2).tolist()
    s_x = ((x_bef + x_af)/2).tolist()
    s_y = ((y_bef + y_af)/2).tolist()
    s_z = ((z_bef + z_af)/2).tolist()

    ratios = np.linspace(0.1, 0.9, num=3)  # 9 evenly spaced points between 0.1 and 0.9

    # Generate sampling points for each coordinate
    s_x_t = [r * x_b + (1 - r) * x_a for r in ratios for x_b, x_a in zip(x_bef, x_af)]
    s_y_t = [r * y_b + (1 - r) * y_a for r in ratios for y_b, y_a in zip(y_bef, y_af)]
    s_z_t = [r * z_b + (1 - r) * z_a for r in ratios for z_b, z_a in zip(z_bef, z_af)]

    x_bef, y_bef, z_bef = d_bef_coords_repeated['x'].values, d_bef_coords_repeated['y'].values, d_bef_coords_repeated['z'].values
    x_af, y_af, z_af = d_af_coords_repeated['x'].values, d_af_coords_repeated['y'].values, d_af_coords_repeated['z'].values

    d_bef_x = x_bef.astype(np.int64).tolist()
    d_bef_y = y_bef.astype(np.int64).tolist()
    d_bef_z = z_bef.astype(np.int64).tolist()

    d_af_x = x_af.astype(np.int64).tolist()
    d_af_y = y_af.astype(np.int64).tolist()
    d_af_z = z_af.astype(np.int64).tolist()

    # Calculate distances using vectorized operations
    d_distances = np.sqrt((x_bef - x_af)**2 + (y_bef - y_af)**2 + (z_bef - z_af)**2).tolist()
    d_x = ((x_bef + x_af)/2).tolist()
    d_y = ((y_bef + y_af)/2).tolist()
    d_z = ((z_bef + z_af)/2).tolist()

    # Generate sampling points for each coordinate
    d_x_t = [r * x_b + (1 - r) * x_a for r in ratios for x_b, x_a in zip(x_bef, x_af)]
    d_y_t = [r * y_b + (1 - r) * y_a for r in ratios for y_b, y_a in zip(y_bef, y_af)]
    d_z_t = [r * z_b + (1 - r) * z_a for r in ratios for z_b, z_a in zip(z_bef, z_af)]

    connection_type_n = ["n"] * len(src)
    
    return src, dst, s_bef, s_bef_x, s_bef_y, s_bef_z, s_af, s_af_x, s_af_y, s_af_z, s_x, s_y, s_z, s_x_t, s_y_t, s_z_t, s_distances, d_bef, d_bef_x, d_bef_y, d_bef_z, d_af, d_af_x, d_af_y, d_af_z, d_x, d_y, d_z, d_x_t, d_y_t, d_z_t , d_distances, connection_type_n, bef_mapping, af_mapping

In [ ]:
# This code combines the previously defined functions and, in the end, creates and returns 
# the dictionary representing the neuron morphology aware graph.
def create_neuron_morphology_aware_graph(id_list):
    #ensure no duplicate ids
    id_list= list(id_list)

    #to remove neurons that failed loading
    id_list_copy= id_list.copy()

    #all of the columns in the new df
    neurons={}
    src, dst, s_bef, s_bef_x, s_bef_y, s_bef_z, s_af, s_af_x, s_af_y, s_af_z, s_x, s_y, s_z, s_x_2, s_y_2, s_z_2, s_x_3, s_y_3, s_z_3, s_x_t, s_y_t, s_z_t, s_distances, d_bef, d_bef_x, d_bef_y, d_bef_z, d_af, d_af_x, d_af_y, d_af_z, d_x, d_y, d_z, d_x_2, d_y_2, d_z_2, d_x_3, d_y_3, d_z_3, d_x_t, d_y_t, d_z_t, d_distances, connection_type_n, n_id = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

    bef_mapping, af_mapping = {}, {}
    for id in id_list:
        #load neuron
        n= get_neuron_local(id, None, 100000)

        if n == None:
             id_list_copy.remove(id)
             continue
        
        neurons[id] = {"n": n}
        edges = n.edges
        #create morphology aware graph without synapses for each neuron
        src_temp, dst_temp, s_bef_temp, s_bef_x_temp, s_bef_y_temp, s_bef_z_temp, s_af_temp, s_af_x_temp, s_af_y_temp, s_af_z_temp, s_x_temp, s_y_temp, s_z_temp, s_x_t_temp, s_y_t_temp, s_z_t_temp, s_distances_temp, d_bef_temp, d_bef_x_temp, d_bef_y_temp, d_bef_z_temp, d_af_temp, d_af_x_temp, d_af_y_temp, d_af_z_temp, d_x_temp, d_y_temp, d_z_temp, d_x_t_temp, d_y_t_temp, d_z_t_temp, d_distances_temp, connection_type_n_temp, bef_mapping, af_mapping = spatial_connectome_creation(edges, n)
        
        neurons[id].update({"src": src_temp, "dst": dst_temp, "s_x": s_x_temp, "s_y": s_y_temp, "s_z": s_z_temp, "s_x": s_x_temp, "s_y": s_y_temp, "s_z": s_z_temp, "s_x_t": s_x_t_temp, "s_y_t": s_y_t_temp, "s_z_t": s_z_t_temp , "d_x": d_x_temp, "d_y": d_y_temp, "d_z": d_z_temp, "d_x_t": d_x_t_temp, "d_y_t": d_y_t_temp, "d_z_t": s_z_t_temp , "bef_mapping": bef_mapping, "af_mapping": af_mapping})
        
        src.extend(src_temp)
        dst.extend(dst_temp)
        s_bef.extend(s_bef_temp)
        s_bef_x.extend(s_bef_x_temp)
        s_bef_y.extend(s_bef_y_temp)
        s_bef_z.extend(s_bef_z_temp)
        s_af.extend(s_af_temp)
        s_af_x.extend(s_af_x_temp)
        s_af_y.extend(s_af_y_temp)
        s_af_z.extend(s_af_z_temp)
        s_x.extend(s_x_temp)
        s_y.extend(s_y_temp)
        s_z.extend(s_z_temp)
        s_distances.extend(s_distances_temp)
        d_bef.extend(d_bef_temp)
        d_bef_x.extend(d_bef_x_temp)
        d_bef_y.extend(d_bef_y_temp)
        d_bef_z.extend(d_bef_z_temp)
        d_af.extend(d_af_temp)
        d_af_x.extend(d_af_x_temp)
        d_af_y.extend(d_af_y_temp)
        d_af_z.extend(d_af_z_temp)
        d_x.extend(d_x_temp)
        d_y.extend(d_y_temp)
        d_z.extend(d_z_temp)
        d_distances.extend(d_distances_temp)
        connection_type_n.extend(connection_type_n_temp)
        n_id.extend([id]* len(src_temp))
    
    if id_list_copy == []:
        return None
    
    #include the synapses to the neuron morphology aware graph
    synaptic_src, synaptic_dst, connection_type_s, x, y, z, n_combo, dists, synaptic_s_bef, synaptic_s_af, synaptic_d_bef, synaptic_d_af= synaptic_data(neurons)

    spatial_connectome_edge_dict = {
    "src": src + dst + synaptic_src, #id of source segment
    "dst": dst + src+  synaptic_dst, #id of destination segment
    "s_bef": s_bef + d_bef+ synaptic_s_bef, #id of starting skeleton of source segment
    "s_bef_x": s_bef_x + d_bef_x+ [0]* len(synaptic_src), #x coordinate of starting skeleton
    "s_bef_y": s_bef_y +d_bef_y+ [0]* len(synaptic_src), #y coordinate of starting skeleton
    "s_bef_z": s_bef_z + d_bef_z+ [0]* len(synaptic_src), #z coordinate of starting skeleton
    "s_af": s_af+ d_af+ synaptic_s_af, #id of ending skeleton of source segment
    "s_af_x": s_af_x+d_af_x+ [0]* len(synaptic_src), #x coordinate of ending skeleton
    "s_af_y": s_af_y+d_af_y+ [0]* len(synaptic_src), #y coordinate of ending skeleton
    "s_af_z": s_af_z+d_af_z+ [0]* len(synaptic_src), #z coordinate of ending skeleton
    "s_x": s_x+d_x+ x, #middle point of source segment
    "s_y": s_y+d_y+ y, #middle point of source segment
    "s_z": s_z+d_z+ z, #middle point of source segment
    "s_distance": s_distances+d_distances+ dists, #length of source segment
    "d_bef": d_bef+s_bef+ synaptic_d_bef, #id of starting skeleton of destination segment
    "d_bef_x": d_bef_x+s_bef_x+ [0]* len(synaptic_src), #x coordinate of starting skeleton
    "d_bef_y": d_bef_y+s_bef_y+ [0]* len(synaptic_src), #y coordinate of starting skeleton
    "d_bef_z": d_bef_z+s_bef_z+ [0]* len(synaptic_src), #z coordinate of starting skeleton
    "d_af": d_af+s_af+ synaptic_d_af, #id of ending skeleton of destination segment
    "d_af_x": d_af_x+s_af_x+ [0]* len(synaptic_src), #x coordinate of ending skeleton
    "d_af_y": d_af_y+s_af_y+ [0]* len(synaptic_src), #y coordinate of ending skeleton
    "d_af_z": d_af_z+s_af_z+ [0]* len(synaptic_src), #z coordinate of ending skeleton
    "d_x": d_x+s_x+ x, #middle point of destination segment
    "d_y": d_y+s_y+ y, #middle point of destination segment
    "d_z": d_z+s_z+ z, #middle point of destination segment
    "d_distance": d_distances+s_distances+ dists, #length of destination segment
    "n_id": n_id +n_id+ n_combo, #id (two ids if synaptic connection) of neuron
    "connection_type": connection_type_n +connection_type_n+ connection_type_s #specifies wether intra or inter neuron connection
    }

    return spatial_connectome_edge_dict

In [4]:
#example code on how to create and store the neuron morphology aware graph given a list of neuron ids
neuron_ids=None #provide neuron ids list
spatial_connectome_edge_dict = create_neuron_morphology_aware_graph(neuron_ids)
spatial_connectome_edge_dict.to_csv("/path_to/spatial_connectome_edge_df.csv", index=False)
spatial_connectome_edge_df = pd.DataFrame(spatial_connectome_edge_dict)